In [106]:
import pandas as pd
from apyori import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

In [107]:
df = pd.read_csv('C:\\Users\\gabri\\OneDrive\\Área de Trabalho\\CienciaDeDados\\atividade_mineracao\\csv\\compras.csv')
df.head()

,cliente,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,JÚLIO DEMARQUE,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAYCON SOARES,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WEMERSON NETO,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOÃO DEMARQUE,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,MAYCON SANTOS,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
#Dropando a coluna cliente
df = df.drop('cliente', axis=1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,limão,peito de peru,manga,cerveja,picanha,carvão,sal,linguiça,mel,geleia,cereal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,salsicha,cerveja,picanha,catchup,geleia,cebola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,queijo,frutas,chá,alface,granola,feijão,farinha de trigo,suco de laranja,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pimenta,sardinha enlatada,barra de cereal,molho de tomate,gelatina,vinho branco,pão,tempero pronto,vinho tinto,presunto,arroz,feijão,NaN,NaN,NaN,NaN,NaN,NaN
4,detergente,filé de frango,arroz,feijão,limão,papel higiênico,presunto,farinha de trigo,salada,alface,cenoura,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
# itens com mais compras
itens_mais_comprados = df.stack().value_counts()
# 20 itens mais comprados
itens_mais_comprados.head(20)

queijo               1241
presunto             1129
pão                  1056
feijão               1040
picanha               869
arroz                 856
salada                837
manteiga              803
cerveja               759
leite                 722
bolo                  716
salsicha              682
açúcar                631
sorvete               621
alface                618
carvão                612
batata frita          607
cerveja artesanal     581
tomate                577
refrigerante          573
Name: count, dtype: int64

In [110]:
# Aplicando o Apriori
transactions = []
for i in range(0,df.shape[0]):
    transactions.append([str(df.values[i,j])for j in range(0,df.shape[1])])
regras = apriori(transactions=transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2, max_length=2)
resultados = list(regras)
resultados

[RelationRecord(items=frozenset({'alface', 'azeite'}), support=0.0104, ordered_statistics=[OrderedStatistic(items_base=frozenset({'azeite'}), items_add=frozenset({'alface'}), confidence=0.20392156862745098, lift=3.3594986594308236)]),
 RelationRecord(items=frozenset({'alface', 'cenoura'}), support=0.0129, ordered_statistics=[OrderedStatistic(items_base=frozenset({'alface'}), items_add=frozenset({'cenoura'}), confidence=0.21252059308072488, lift=3.885202798550729), OrderedStatistic(items_base=frozenset({'cenoura'}), items_add=frozenset({'alface'}), confidence=0.23583180987202926, lift=3.885202798550729)]),
 RelationRecord(items=frozenset({'salada', 'alface'}), support=0.0254, ordered_statistics=[OrderedStatistic(items_base=frozenset({'alface'}), items_add=frozenset({'salada'}), confidence=0.4184514003294893, lift=5.096850186717287), OrderedStatistic(items_base=frozenset({'salada'}), items_add=frozenset({'alface'}), confidence=0.3093788063337393, lift=5.096850186717287)]),
 RelationRecor

In [111]:
# Aplicando o FP-Growth
df_compras_lista = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

te = TransactionEncoder()
te_ary = te.fit(df_compras_lista).transform(df_compras_lista)

# Converte o array resultante em um DataFrame pandas
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Agora você pode aplicar o fpgrowth no DataFrame one-hot
frequent_itemsets = fpgrowth(df_onehot, min_support=0.03,use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=3)

# imprimindo as regras mais importantes ordenadas por lift
rules = rules.sort_values(by='lift', ascending=False)
rules.head(30)









#Aplicando one-hot encoding
#df_encoded = pd.get_dummies(df)

# Aplicar o FP-Growth para encontrar conjuntos de itens frequentes
#frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)

#print(frequent_itemsets)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,(cerveja),(carvão),0.0754,0.0604,0.0407,0.539788,8.936884,0.036146,2.041667,0.960528
3,(carvão),(cerveja),0.0604,0.0754,0.0407,0.673841,8.936884,0.036146,2.834814,0.945194
24,(frango),(arroz),0.0566,0.0852,0.0425,0.750883,8.813185,0.037678,3.672176,0.939722
25,(arroz),(frango),0.0852,0.0566,0.0425,0.498826,8.813185,0.037678,1.882381,0.969101
4,(carvão),(picanha),0.0604,0.0856,0.0397,0.657285,7.678560,0.034530,2.668104,0.925678
5,(picanha),(carvão),0.0856,0.0604,0.0397,0.463785,7.678560,0.034530,1.752282,0.951189
27,(feijão),(frango),0.1026,0.0566,0.0363,0.353801,6.250904,0.030493,1.459922,0.936063
26,(frango),(feijão),0.0566,0.1026,0.0363,0.641343,6.250904,0.030493,2.502110,0.890421
19,(feijão),(arroz),0.1026,0.0852,0.0544,0.530214,6.223174,0.045658,1.947271,0.935269
18,(arroz),(feijão),0.0852,0.1026,0.0544,0.638498,6.223174,0.045658,2.482418,0.917480
